In [1]:
import pandas as pd
import json
import os

import warnings 
warnings.filterwarnings("ignore")
from importlib import reload

# this ensures that I can update the class without losing my variables in my notebook
import xenium_cluster
reload(xenium_cluster)
from xenium_cluster import XeniumCluster
from utils.metrics import *

In [2]:
# Path to your .gz file
file_path = 'data/hBreast/transcripts.csv.gz'

# Read the gzipped CSV file into a DataFrame
df_transcripts = pd.read_csv(file_path, compression='gzip')

# drop cells without ids
df_transcripts = df_transcripts[df_transcripts["cell_id"] != -1]

# drop blanks and controls
df_transcripts = df_transcripts[~df_transcripts["feature_name"].str.startswith('BLANK_') & ~df_transcripts["feature_name"].str.startswith('NegControl')]

In [3]:
def run_experiment(data, dataset_name: str, current_spot_size: int, third_dim: bool, resolutions: list):
    
    clustering = XeniumCluster(data=data, dataset_name=dataset_name)
    clustering.set_spot_size(current_spot_size)
    clustering.create_spot_data(third_dim=third_dim, save_data=True)

    print(f"The size of the spot data is {clustering.xenium_spot_data.shape}")

    clustering.normalize_counts(clustering.xenium_spot_data)
    clustering.generate_neighborhood_graph(clustering.xenium_spot_data, plot_pcas=False)

    Leiden_cluster = clustering.Leiden(clustering.xenium_spot_data, resolutions, embedding="umap", save_plot=True)

    return clustering, Leiden_cluster

In [4]:
import numpy as np
import torch
from scipy.spatial.distance import cdist

def record_results(original_data, cluster_dict, results_dir, model_name, filename, spot_size, third_dim, K=None, resolution=None, uses_spatial=True):

    if resolution is not None:
        current_clustering = np.array(cluster_dict[model_name][spot_size][third_dim].get(
            resolution, 
            cluster_dict[model_name][spot_size][third_dim]
        ))
    else:
        current_clustering = np.array(cluster_dict[model_name][spot_size][third_dim][uses_spatial].get(
            K, 
            cluster_dict[model_name][spot_size][third_dim][uses_spatial]
        ))
    cluster_labels = np.unique(current_clustering)

    original_data.xenium_spot_data.obs[f"{model_name} cluster"] = np.array(current_clustering)
    dirpath = f"{results_dir}/{model_name}{'/' + (str(resolution) if resolution is not None else str(K))}/clusters/{spot_size}"
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

    filepath = f"{dirpath}/{filename}{resolution}.csv"

    original_data.xenium_spot_data.obs[f"{model_name} cluster"].to_csv(filepath)
    # Extracting row, col, and cluster values from the dataframe+978
    rows = torch.tensor(original_data.xenium_spot_data.obs["row"].astype(int))
    cols = torch.tensor(original_data.xenium_spot_data.obs["col"].astype(int))
    clusters = torch.tensor(original_data.xenium_spot_data.obs[f"{model_name} cluster"].astype(int))
    cluster_labels = np.unique(clusters)

    num_rows = int(max(rows) - min(rows) + 1)
    num_cols = int(max(cols) - min(cols) + 1)

    cluster_grid = torch.zeros((num_rows, num_cols), dtype=torch.int)

    cluster_grid[rows, cols] = torch.tensor(clusters, dtype=torch.int)

    mpd = {}
    for label in cluster_labels:
        current_cluster_locations = torch.stack(torch.where((cluster_grid == label)), axis=1).to(float)
        mpd[f"Cluster {label}"] = spot_size * torch.mean(torch.cdist(current_cluster_locations, current_cluster_locations)).item()
        print(f"POSSIBLE {len(cluster_labels)}", label, mpd[f"Cluster {label}"])

    mpd_dirpath = f"{results_dir}/{model_name}{'/' + (str(resolution) if resolution is not None else str(K))}/mpd/{spot_size}/"
    if not os.path.exists(mpd_dirpath):
        os.makedirs(mpd_dirpath)

    mpd_filepath = f"{mpd_dirpath}/{filename}_mpd.json"
    with open(mpd_filepath, "w") as f:
        json.dump(mpd, f, indent=4)

In [5]:
cluster_dict = {"Leiden": {}}
mpd = {"Leiden": {}}
results_dir = "results/hBreast"

In [6]:
import matplotlib
matplotlib.use('Agg')

resolutions = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0]
for spot_size in [50, 75, 100]:
    for third_dim in [False]:
        cluster_results_filename = f"clusters_RES="
        original_data, Leiden_cluster = run_experiment(df_transcripts, "hBreast", spot_size, third_dim, resolutions)

        # Leiden
        if "Leiden" not in cluster_dict:
            cluster_dict["Leiden"] = {}
        if spot_size not in cluster_dict["Leiden"]:
            cluster_dict["Leiden"][spot_size] = {}
        cluster_dict["Leiden"][spot_size][third_dim] = {res: clusters.tolist() for res, clusters in Leiden_cluster.items()}
        for resolution in resolutions:
            record_results(original_data, cluster_dict, results_dir, "Leiden", cluster_results_filename, spot_size, third_dim, resolution=resolution)

        print(f"Cluster with spot size {(spot_size, third_dim)} completed.")

The size of the spot data is (23444, 280)
POSSIBLE 2 0 4745.378529640296
POSSIBLE 2 1 4174.6694933615845
POSSIBLE 6 0 4862.628847969457
POSSIBLE 6 1 3839.921423828178
POSSIBLE 6 2 3784.754257378247
POSSIBLE 6 3 4029.9394320342876
POSSIBLE 6 4 1531.038732603824
POSSIBLE 6 5 1044.2212386844667
POSSIBLE 13 0 5396.889179161116
POSSIBLE 13 1 3592.8103653932917
POSSIBLE 13 2 3517.8020029831205
POSSIBLE 13 3 2913.5690605605264
POSSIBLE 13 4 3844.330781761132
POSSIBLE 13 5 2528.0697666895257
POSSIBLE 13 6 4224.424008289204
POSSIBLE 13 7 1604.6066532351979
POSSIBLE 13 8 3764.619218930621
POSSIBLE 13 9 3778.106622516302
POSSIBLE 13 10 1172.939298884277
POSSIBLE 13 11 3972.2749165975115
POSSIBLE 13 12 3188.7067234396222
POSSIBLE 17 0 5553.476509865308
POSSIBLE 17 1 2621.858713919441
POSSIBLE 17 2 3569.5863115778166
POSSIBLE 17 3 4232.719011353107
POSSIBLE 17 4 3741.2394429306764
POSSIBLE 17 5 3471.3840722519003
POSSIBLE 17 6 2839.0363344459042
POSSIBLE 17 7 2864.5482783672232
POSSIBLE 17 8 3753.5

In [7]:
spot_sizes = [50,75,100]
resolutions = [0.25, 0.5, 0.75, 1.0]
in_billions = 1_000_000_000
method = "Leiden"
for spot_size in spot_sizes:
    for K in [17]:
        for resolution in resolutions:
            filename = f"results/hBreast/{method}/{resolution}/mpd/{spot_size}/{cluster_results_filename}_mpd.json"
            if os.path.exists(filename):
                with open(filename, "r") as mpd_dict:
                    current_mpd = json.load(mpd_dict)
                print("Method:", method, "Spot Size", spot_size, "Resolution", resolution, "Num Clusters:", len(current_mpd), "Total mpd", sum(current_mpd.values()) / in_billions)

Method: Leiden Spot Size 50 Resolution 0.25 Num Clusters: 6 Total mpd 1.9092503932498462e-05
Method: Leiden Spot Size 50 Resolution 0.5 Num Clusters: 13 Total mpd 4.3499148598441445e-05
Method: Leiden Spot Size 50 Resolution 0.75 Num Clusters: 17 Total mpd 5.593160193262027e-05
Method: Leiden Spot Size 50 Resolution 1.0 Num Clusters: 20 Total mpd 6.524583595322188e-05
Method: Leiden Spot Size 75 Resolution 0.25 Num Clusters: 5 Total mpd 1.9226184107391796e-05
Method: Leiden Spot Size 75 Resolution 0.5 Num Clusters: 11 Total mpd 3.526911220267226e-05
Method: Leiden Spot Size 75 Resolution 0.75 Num Clusters: 14 Total mpd 4.5217163532891436e-05
Method: Leiden Spot Size 75 Resolution 1.0 Num Clusters: 17 Total mpd 5.360656117048983e-05
Method: Leiden Spot Size 100 Resolution 0.25 Num Clusters: 6 Total mpd 2.3271593834089e-05
Method: Leiden Spot Size 100 Resolution 0.5 Num Clusters: 9 Total mpd 2.881843232376044e-05
Method: Leiden Spot Size 100 Resolution 0.75 Num Clusters: 14 Total mpd 4.5